In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, SelectMultiple, fixed


In [3]:
DATA_PATH = r"C:\F1_analysis\data\processed\laps_2024_cleaned.csv"
df = pd.read_csv(DATA_PATH)
df.head(2)

,index,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,Round,Race_Name,Circuit,Country,LapTimeSeconds
0,0,0 days 01:01:37.489000,VER,1,0 days 00:01:37.284000,1.0,1.0,NaN,NaN,False,SOFT,4.0,False,Red Bull Racing,1,Bahrain Grand Prix,Bahrain Grand Prix,Bahrain,97.284
1,1,0 days 01:03:13.785000,VER,1,0 days 00:01:36.296000,2.0,1.0,NaN,NaN,True,SOFT,5.0,False,Red Bull Racing,1,Bahrain Grand Prix,Bahrain Grand Prix,Bahrain,96.296


In [4]:
def get_drivers(race_name):
    return sorted(df[df["Race_Name"] == race_name]["Driver"].unique())

In [5]:
def plot_lap_times(selected_race, selected_drivers):
    race_df = df[df["Race_Name"] == selected_race]

    fig, ax = plt.subplots(figsize=(12, 6))

    # Assign a color to each team
    teams = race_df["Team"].unique()
    team_colors = dict(zip(teams, sns.color_palette("tab10", len(teams))))

    for driver in selected_drivers:
        driver_laps = race_df[race_df["Driver"] == driver]
        team = driver_laps["Team"].iloc[0]
        ax.plot(
            driver_laps["LapNumber"],
            driver_laps["LapTimeSeconds"],
            label=driver,
            color=team_colors[team],
            marker='o'
        )

        # Optional: mark pit stops
        pit_laps = driver_laps[driver_laps["PitInTime"].notna()]
        for lap in pit_laps["LapNumber"]:
            ax.axvline(lap, color=team_colors[team], linestyle='--', alpha=0.5)

    ax.invert_yaxis()
    ax.set_xlabel("Lap Number")
    ax.set_ylabel("Lap Time (s)")
    ax.set_title(f"Lap Times Overlay – {selected_race}")
    ax.grid(True)
    ax.legend()
    plt.show()


In [8]:
race_options = sorted(df["Race_Name"].unique())

@interact(selected_race=race_options)
def select_drivers(selected_race):
    drivers = get_drivers(selected_race)
    interact(
        plot_lap_times,
        selected_race=fixed(selected_race),
        selected_drivers=SelectMultiple(
            options=drivers,
            value=drivers[:2],  # default 2 drivers
            description="Drivers",
            rows=5
        )
    )

interactive(children=(Dropdown(description='selected_race', options=('Abu Dhabi Grand Prix', 'Australian Grand…

In [9]:
def get_drivers(race_name):
    return sorted(df[df["Race_Name"] == race_name]["Driver"].unique())

def plot_tyre_degradation(selected_race, selected_drivers):
    race_df = df[df["Race_Name"] == selected_race]
    fig, ax = plt.subplots(figsize=(12, 6))

    teams = race_df["Team"].unique()
    team_colors = dict(zip(teams, sns.color_palette("tab10", len(teams))))

    for driver in selected_drivers:
        driver_laps = race_df[race_df["Driver"] == driver].sort_values("LapNumber").reset_index()
        # Calculate lap delta as tire degradation
        driver_laps['LapDelta'] = driver_laps['LapTimeSeconds'].diff().fillna(0)
        # Optional: cumulative degradation from first lap
        driver_laps['CumulativeDegradation'] = driver_laps['LapDelta'].cumsum()

        ax.plot(
            driver_laps["LapNumber"],
            driver_laps["CumulativeDegradation"],
            label=driver,
            color=team_colors[driver_laps["Team"].iloc[0]],
            marker='o'
        )

        # Optional: mark pit stops
        pit_laps = driver_laps[driver_laps["PitInTime"].notna()]
        for lap in pit_laps["LapNumber"]:
            ax.axvline(lap, color=team_colors[driver_laps["Team"].iloc[0]], linestyle='--', alpha=0.5)

    ax.set_xlabel("Lap Number")
    ax.set_ylabel("Cumulative Lap Time Increase (s)")
    ax.set_title(f"Tire Degradation – {selected_race}")
    ax.grid(True)
    ax.legend()
    plt.show()

In [10]:
race_options = sorted(df["Race_Name"].unique())

@interact(selected_race=race_options)
def select_drivers(selected_race):
    drivers = get_drivers(selected_race)
    interact(
        plot_tyre_degradation,
        selected_race=fixed(selected_race),
        selected_drivers=SelectMultiple(
            options=drivers,
            value=drivers[:2],  # default 2 drivers
            description="Drivers",
            rows=5
        )
    )

interactive(children=(Dropdown(description='selected_race', options=('Abu Dhabi Grand Prix', 'Australian Grand…